In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
# 코드에서 nms제외해야함

In [1]:
# !python3 -m pip install --pre --upgrade tensorrt
# !apt-get install tensorrt -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cuda-cccl-12-1 cuda-cccl-12-4 cuda-cudart-12-1 cuda-cudart-12-4
  cuda-cudart-dev-12-1 cuda-cudart-dev-12-4 cuda-driver-dev-12-1
  cuda-driver-dev-12-4 cuda-nvcc-12-1 cuda-toolkit-12-1-config-common
  cuda-toolkit-12-4-config-common cuda-toolkit-12-config-common libnvinfer-bin
  libnvinfer-dev libnvinfer-dispatch-dev libnvinfer-dispatch10
  libnvinfer-headers-dev libnvinfer-headers-plugin-dev libnvinfer-lean-dev
  libnvinfer-lean10 libnvinfer-plugin-dev libnvinfer-plugin10
  libnvinfer-samples libnvinfer-vc-plugin-dev libnvinfer-vc-plugin10
  libnvinfer10 libnvonnxparsers-dev libnvonnxparsers10 python3-libnvinfer
  python3-libnvinfer-dev python3-libnvinfer-dispatch python3-libnvinfer-lean
The following NEW packages will be installed:
  cuda-cccl-12-1 cuda-cccl-12-4 cuda-cudart-12-1 cuda-cudart-12-4
  cuda-cudart-dev-12-1 cuda-cudart-dev-

In [ ]:
base_name = "./deploy_pillar_sfaw_3d/dsvt_3d_bb.onnx"

!trtexec --onnx=./deploy_files/dsvt.onnx  --saveEngine=./deploy_files/dsvt.engine \
--memPoolSize=workspace:4096 --verbose --buildOnly --device=1 --fp16 \
--tacticSources=+CUDNN,+CUBLAS,-CUBLAS_LT,+EDGE_MASK_CONVOLUTIONS \
--minShapes=src:3000x192,set_voxel_inds_tensor_shift_0:2x170x36,set_voxel_inds_tensor_shift_1:2x100x36,set_voxel_masks_tensor_shift_0:2x170x36,set_voxel_masks_tensor_shift_1:2x100x36,pos_embed_tensor:4x2x3000x192 \
--optShapes=src:20000x192,set_voxel_inds_tensor_shift_0:2x1000x36,set_voxel_inds_tensor_shift_1:2x700x36,set_voxel_masks_tensor_shift_0:2x1000x36,set_voxel_masks_tensor_shift_1:2x700x36,pos_embed_tensor:4x2x20000x192 \
--maxShapes=src:35000x192,set_voxel_inds_tensor_shift_0:2x1500x36,set_voxel_inds_tensor_shift_1:2x1200x36,set_voxel_masks_tensor_shift_0:2x1500x36,set_voxel_masks_tensor_shift_1:2x1200x36,pos_embed_tensor:4x2x35000x192 \
> debug.log 2>&1

In [2]:
import tensorrt

In [2]:
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
import os
import numpy as np
import torch
# # import onnx
import onnxruntime as ort
import torch.nn as nn

# from typing import Sequence, NamedTuple



In [8]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [3]:
# cfg_file = "./onnx_config.yaml"
cfg_file = "./pillar/config.yaml"
cfg_from_yaml_file(cfg_file, cfg)
if os.path.exists('./deploy_pillar_sfaw_all')==False:
    os.mkdir('./deploy_pillar_sfaw_all')
log_file = './deploy_pillar_sfaw_all/log_trt.log'
logger = common_utils.create_logger(log_file, rank=0)
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=1,
    dist=False, workers=8, logger=logger, training=False
)

2024-04-11 09:27:54,721   INFO  Loading Custom dataset.
2024-04-11 09:27:54,722   INFO  Total samples for CUSTOM dataset: 0


In [4]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
ckpt = "./pillar/model.pth"
model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False, pre_trained_path=None)
model.eval()
model.cuda()

2024-04-11 09:28:02,181   INFO  ==> Loading parameters from checkpoint ./pillar/model.pth to GPU
2024-04-11 09:28:02,574   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f
2024-04-11 09:28:02,644   INFO  ==> Done (loaded 391/391)


CenterPoint(
  (vfe): DynamicVoxelVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayerV2(
        (linear): Linear(in_features=10, out_features=96, bias=False)
        (norm): BatchNorm1d(96, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): PFNLayerV2(
        (linear): Linear(in_features=192, out_features=192, bias=False)
        (norm): BatchNorm1d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
    )
  )
  (backbone_3d): DSVT(
    (input_layer): DSVTInputLayer(
      (posembed_layers): ModuleList(
        (0): ModuleList(
          (0-3): 4 x ModuleList(
            (0-1): 2 x PositionEmbeddingLearned(
              (position_embedding_head): Sequential(
                (0): Linear(in_features=2, out_features=192, bias=True)
                (1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
       

In [5]:
batch_dict = torch.load("/mnt/nas2/users/eslim/onnx/sfaw/input_dict.pth", map_location="cuda")
inputs = batch_dict


In [6]:
model

CenterPoint(
  (vfe): DynamicVoxelVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayerV2(
        (linear): Linear(in_features=10, out_features=96, bias=False)
        (norm): BatchNorm1d(96, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): PFNLayerV2(
        (linear): Linear(in_features=192, out_features=192, bias=False)
        (norm): BatchNorm1d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
    )
  )
  (backbone_3d): DSVT(
    (input_layer): DSVTInputLayer(
      (posembed_layers): ModuleList(
        (0): ModuleList(
          (0-3): 4 x ModuleList(
            (0-1): 2 x PositionEmbeddingLearned(
              (position_embedding_head): Sequential(
                (0): Linear(in_features=2, out_features=192, bias=True)
                (1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
       

In [13]:
# 3D BB

In [9]:
base_name = "./deploy_pillar_sfaw_3d/dsvt_3d_bb"
onnx_path = f"{base_name}.onnx"
ort_session = ort.InferenceSession(onnx_path)



In [10]:
with torch.no_grad():
    DSVT_Backbone = model.backbone_3d
    dsvtblocks_list = DSVT_Backbone.stage_0
    layer_norms_list = DSVT_Backbone.residual_norm_stage_0
    inputs = model.vfe(inputs)
    voxel_info = DSVT_Backbone.input_layer(inputs)
    set_voxel_inds_list = [[voxel_info[f'set_voxel_inds_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    set_voxel_masks_list = [[voxel_info[f'set_voxel_mask_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    pos_embed_list = [[[voxel_info[f'pos_embed_stage{s}_block{b}_shift{i}'] for i in range(2)] for b in range(4)] for s in range(1)]
    pillar_features = inputs['voxel_features']


In [11]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(pillar_features),
              ort_session.get_inputs()[1].name: to_numpy(set_voxel_inds_list[0][0]),
              ort_session.get_inputs()[2].name: to_numpy(set_voxel_inds_list[0][1]),
              ort_session.get_inputs()[3].name: to_numpy(set_voxel_masks_list[0][0]),
              ort_session.get_inputs()[4].name: to_numpy(set_voxel_masks_list[0][1]),
              ort_session.get_inputs()[5].name: to_numpy(torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0)),}
ort_outs = ort_session.run(None, ort_inputs)

In [12]:
ort_outs

[array([[-1.09687102e+00, -8.87880400e-02,  1.56932503e-01, ...,
         -1.04772103e+00,  3.80000025e-02, -1.98598042e-01],
        [-1.09640002e+00, -7.08313286e-02,  1.60521924e-01, ...,
         -1.08235085e+00,  4.23637405e-02, -1.92351431e-01],
        [-1.08620584e+00, -1.00942045e-01,  1.78571254e-01, ...,
         -1.05504239e+00,  2.63126958e-02, -1.59484476e-01],
        ...,
        [-3.60771865e-01,  2.12830842e-01, -1.58340424e-01, ...,
         -1.23321116e+00, -1.63467184e-01,  1.67723268e-01],
        [-6.18592083e-01,  3.51619005e-01, -1.11299597e-01, ...,
         -1.37231803e+00, -1.09309152e-01, -5.45044169e-02],
        [-6.13398969e-01,  3.21275711e-01, -1.22284256e-02, ...,
         -1.24491286e+00,  6.93649054e-04, -1.37990996e-01]], dtype=float32)]

In [ ]:
## BEV

In [36]:
input_dict = {}
input_dict['batch_dict']={}
input_dict['batch_dict']['voxel_features'] = ort_outs[0]
input_dict['batch_dict']['use_lead_xyz'] = inputs['use_lead_xyz']
input_dict['batch_dict']['batch_size'] = torch.tensor(inputs['batch_size'])
input_dict['batch_dict']['pillar_features'] = ort_outs[0]
input_dict['batch_dict']['voxel_coords'] = inputs['voxel_coords']


In [41]:
base_name = "./deploy_pillar_sfaw_bev/dsvt_bev"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"

ort_session = ort.InferenceSession(onnx_path)

ort_inputs = {
    'voxel_features' : input_dict['batch_dict']['voxel_features'],
    'use_lead_xyz' : to_numpy(input_dict['batch_dict']['use_lead_xyz']),
    'batch_size' :  np.array(1, dtype=np.int64),
    'pillar_features' : input_dict['batch_dict']['pillar_features'],
    'voxel_coords' : to_numpy(input_dict['batch_dict']['voxel_coords']).astype(np.int64),
}
ort_outs = ort_session.run(None, ort_inputs)
ort_outs

[array([[-1.09687102e+00, -8.87880400e-02,  1.56932503e-01, ...,
         -1.04772103e+00,  3.80000025e-02, -1.98598042e-01],
        [-1.09640002e+00, -7.08313286e-02,  1.60521924e-01, ...,
         -1.08235085e+00,  4.23637405e-02, -1.92351431e-01],
        [-1.08620584e+00, -1.00942045e-01,  1.78571254e-01, ...,
         -1.05504239e+00,  2.63126958e-02, -1.59484476e-01],
        ...,
        [-3.60771865e-01,  2.12830842e-01, -1.58340424e-01, ...,
         -1.23321116e+00, -1.63467184e-01,  1.67723268e-01],
        [-6.18592083e-01,  3.51619005e-01, -1.11299597e-01, ...,
         -1.37231803e+00, -1.09309152e-01, -5.45044169e-02],
        [-6.13398969e-01,  3.21275711e-01, -1.22284256e-02, ...,
         -1.24491286e+00,  6.93649054e-04, -1.37990996e-01]], dtype=float32),
 array([1.], dtype=float32),
 array(1, dtype=int64),
 array([[-1.09687102e+00, -8.87880400e-02,  1.56932503e-01, ...,
         -1.04772103e+00,  3.80000025e-02, -1.98598042e-01],
        [-1.09640002e+00, -7.083132

In [ ]:
## 2D BB

In [44]:
input_dict = {}
input_dict['data_dict']={}
input_dict['data_dict']['voxel_features'] = ort_outs[0]
input_dict['data_dict']['use_lead_xyz'] = ort_outs[1]
input_dict['data_dict']['batch_size'] = ort_outs[2]
input_dict['data_dict']['pillar_features'] = ort_outs[3]
input_dict['data_dict']['voxel_coords'] = ort_outs[4]
input_dict['data_dict']['spatial_features'] = ort_outs[5]


In [45]:
base_name = "./deploy_pillar_sfaw_2d/dsvt_2d_bb"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"

ort_session = ort.InferenceSession(onnx_path)

ort_inputs = {
    'voxel_features' : input_dict['data_dict']['voxel_features'],
    'use_lead_xyz' : input_dict['data_dict']['use_lead_xyz'],
    'batch_size' :  input_dict['data_dict']['batch_size'],
    'pillar_features' : input_dict['data_dict']['pillar_features'],
    'voxel_coords' : input_dict['data_dict']['voxel_coords'],
    'spatial_features' : input_dict['data_dict']['spatial_features']
}
ort_outs = ort_session.run(None, ort_inputs)
ort_outs


[array([[-1.09687102e+00, -8.87880400e-02,  1.56932503e-01, ...,
         -1.04772103e+00,  3.80000025e-02, -1.98598042e-01],
        [-1.09640002e+00, -7.08313286e-02,  1.60521924e-01, ...,
         -1.08235085e+00,  4.23637405e-02, -1.92351431e-01],
        [-1.08620584e+00, -1.00942045e-01,  1.78571254e-01, ...,
         -1.05504239e+00,  2.63126958e-02, -1.59484476e-01],
        ...,
        [-3.60771865e-01,  2.12830842e-01, -1.58340424e-01, ...,
         -1.23321116e+00, -1.63467184e-01,  1.67723268e-01],
        [-6.18592083e-01,  3.51619005e-01, -1.11299597e-01, ...,
         -1.37231803e+00, -1.09309152e-01, -5.45044169e-02],
        [-6.13398969e-01,  3.21275711e-01, -1.22284256e-02, ...,
         -1.24491286e+00,  6.93649054e-04, -1.37990996e-01]], dtype=float32),
 array([1.], dtype=float32),
 array(1, dtype=int64),
 array([[-1.09687102e+00, -8.87880400e-02,  1.56932503e-01, ...,
         -1.04772103e+00,  3.80000025e-02, -1.98598042e-01],
        [-1.09640002e+00, -7.083132

In [46]:
#### Head

In [48]:
# data_dict = {}
input_dict = {}
input_dict['data_dict']={}
input_dict['data_dict']['voxel_features'] = ort_outs[0]
input_dict['data_dict']['use_lead_xyz'] = ort_outs[1]
input_dict['data_dict']['batch_size'] = ort_outs[2]
input_dict['data_dict']['pillar_features'] = ort_outs[3]
input_dict['data_dict']['voxel_coords'] = ort_outs[4]
input_dict['data_dict']['spatial_features'] = ort_outs[5]
input_dict['data_dict']['spatial_features_2d'] = ort_outs[6]



In [50]:
# # test onnx
base_name = "./deploy_pillar_sfaw_head/dsvt_head"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"


ort_session = ort.InferenceSession(onnx_path)

ort_inputs = {
    'voxel_features' : input_dict['data_dict']['voxel_features'],
    'use_lead_xyz' : input_dict['data_dict']['use_lead_xyz'],
    'batch_size' :  input_dict['data_dict']['batch_size'],
    'pillar_features' : input_dict['data_dict']['pillar_features'],
    'voxel_coords' : input_dict['data_dict']['voxel_coords'],
    'spatial_features' : input_dict['data_dict']['spatial_features'],
    'spatial_features_2d' : input_dict['data_dict']['spatial_features_2d']
}

ort_outs = ort_session.run(None, ort_inputs)
ort_outs

ValueError: Required inputs (['onnx::Identity_0', 'onnx::Identity_1', 'onnx::Identity_2', 'onnx::Identity_3', 'onnx::Identity_4', 'onnx::Identity_5', 'input.1']) are missing from input feed (['voxel_features', 'use_lead_xyz', 'batch_size', 'pillar_features', 'voxel_coords', 'spatial_features', 'spatial_features_2d']).